<a href="https://colab.research.google.com/github/Akshayextreme/Fake_news_detection_hackathon/blob/master/Fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 890kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 9.5MB/s 
     |████████████████████████████████| 3.0MB 19.9MB/s 
     |████████████████████████████████| 890kB 32.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6df2b0422516f3c0809f70c45c16d7115e8f9fa4f378d111bbbfa4b938feafc9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
#from transformers import DistilBertTokenizer, DistilBertModel, DistilBertConfig
from transformers import DistilBertForSequenceClassification
from transformers import BertForSequenceClassification, BertTokenizer

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [5]:
df = pd.read_csv('Train.csv')

In [6]:
df.head()

,Labels,Text,Text_Tag
0,1,Says the Annies List political group supports ...,abortion
1,2,When did the decline of coal start? It started...,"energy,history,job-accomplishments"
2,3,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy
3,1,Health care reform legislation is likely to ma...,health-care
4,2,The economic turnaround started at the end of ...,"economy,jobs"


In [7]:
df.isnull().sum()

Labels      0
Text        0
Text_Tag    2
dtype: int64

In [8]:
df.Text_Tag.fillna('Not Available', inplace=True)

In [9]:
df['Text_Tag'] = df.Text_Tag.str.replace(",", " ")

In [169]:
# df['Text'] = df['Text'] + df['Text_Tag']

BERT Classifier

In [10]:
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [11]:
def leng_txt(x):
    in_ids = tokenizer.encode(x, add_special_tokens=True)
    return len(in_ids)

In [12]:
df['length'] = df.Text.apply(leng_txt)

Token indices sequence length is longer than the specified maximum sequence length for this model (712 > 512). Running this sequence through the model will result in indexing errors


In [13]:
df.length.quantile(0.95)

42.0

In [14]:
df.length.median()

22.0

In [15]:
MAX_LEN = 50
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 32
LEARNING_RATE = 1e-05

In [16]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.data = dataframe
        self.text = dataframe.Text
        #self.text_tag = dataframe.Text_Tag
        self.targets = dataframe.Labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        text = str(self.text[idx])
        text = " ".join(text.split())

        #text_tag = str(self.text_tag[idx])

        inputs = self.tokenizer.encode_plus(
            text,
            None,#text_tag,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[idx], dtype=torch.long)
        }

In [17]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=648)
val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
validation_set = CustomDataset(val_dataset, tokenizer, MAX_LEN)

FULL Dataset: (10240, 4)
TRAIN Dataset: (8192, 4)
TEST Dataset: (2048, 4)


In [18]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 4
                }

training_loader = DataLoader(training_set, **train_params)
validation_loader = DataLoader(validation_set, **test_params)

In [176]:
# class DistilBertClass(torch.nn.Module):
#     def __init__(self):
#         super(DistilBertClass, self).__init__()
#         self.l1 = DistilBertModel.from_pretrained('distilbert-base-uncased')
#         self.l2 = torch.nn.Dropout(0.3)
#         self.l3 = torch.nn.Linear(768, 6)

#     def forward(self, ids, mask, token_type_id):
#         output_1= self.l1(ids, attention_mask = mask)#, token_type_ids = token_type_ids)
#         output_2 = self.l2(output_1[0][:,0,:])
#         output = self.l3(output_2)
#         return output

# model = DistilBertClass()
# model.to(device)

In [177]:
# model = DistilBertForSequenceClassification.from_pretrained(
#     "distilbert-base-uncased",
#     num_labels = 6,
#     output_attentions = False,
#     output_hidden_states = False,
# )

# # Tell pytorch to run this model on the GPU.
# model.to(device)

In [19]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = 6,
    output_attentions = False,
    output_hidden_states = False,
)

# Tell pytorch to run this model on the GPU.
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [20]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [21]:
from transformers import get_linear_schedule_with_warmup
EPOCHS = 5
total_steps = len(training_loader) * EPOCHS
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [22]:
# def loss_fn(outputs, targets):
#     return torch.nn.CrossEntropyLoss()(outputs, targets)

In [23]:
def train(epoch):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for i, data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        #outputs = model(ids, mask, token_type_ids)
        # loss, outputs = model(ids, mask, labels=targets)
        loss, outputs = model(ids, mask, token_type_ids, labels=targets)
        _, preds = torch.max(outputs, 1)

        optimizer.zero_grad()
        #loss = loss_fn(outputs, targets)
        running_loss += loss.item()
        running_corrects += torch.sum(preds == targets.data)
        if i % 100 == 0 and not i == 0:
            print(f'Epoch: {epoch}, Loss:  {running_loss / 100}')
            running_loss = 0.0
        
        optimizer.zero_grad()
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
    epoch_acc = running_corrects.double() / training_set.__len__()
    print(f'Train -> Epoch: {epoch}, Accuracy: {epoch_acc}')

In [24]:
def validation(epoch):
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for _, data in enumerate(validation_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.long)

            #outputs = model(ids, mask, token_type_ids)
            # loss, outputs = model(ids, mask, labels=targets)
            loss, outputs = model(ids, mask, token_type_ids, labels=targets)
            _, preds = torch.max(outputs, 1)

            #loss = loss_fn(outputs, targets)
            running_loss += loss.item()
            running_corrects += torch.sum(preds == targets.data)
        
        epoch_loss = running_loss / validation_set.__len__()
        epoch_acc = running_corrects.double() / validation_set.__len__()
        print(f'Valid -> Epoch: {epoch}, Loss: {epoch_loss}, Accuracy: {epoch_acc}')
        return epoch_acc

In [25]:
# EPOCHS = 4
import copy
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(EPOCHS):
    train(epoch)
    epoch_acc = validation(epoch)
    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())

model.load_state_dict(best_model_wts)

Epoch: 0, Loss:  1.7846557307243347
Train -> Epoch: 0, Accuracy: 0.211181640625
Valid -> Epoch: 0, Loss: 0.05382832366740331, Accuracy: 0.24169921875
Epoch: 1, Loss:  1.7240129232406616
Train -> Epoch: 1, Accuracy: 0.2547607421875
Valid -> Epoch: 1, Loss: 0.053218944231048226, Accuracy: 0.2548828125
Epoch: 2, Loss:  1.693709580898285
Train -> Epoch: 2, Accuracy: 0.279541015625
Valid -> Epoch: 2, Loss: 0.053045045235194266, Accuracy: 0.2529296875
Epoch: 3, Loss:  1.6575748217105866
Train -> Epoch: 3, Accuracy: 0.3017578125
Valid -> Epoch: 3, Loss: 0.05315730132861063, Accuracy: 0.24853515625
Epoch: 4, Loss:  1.6343337070941926
Train -> Epoch: 4, Accuracy: 0.3206787109375
Valid -> Epoch: 4, Loss: 0.053205663920380175, Accuracy: 0.24609375


<All keys matched successfully>

In [155]:
df_test = pd.read_csv('Test.csv')
df_test['Labels'] = 0

In [156]:
df_test['Text_Tag'] = df_test.Text_Tag.str.replace(",", " ")

In [155]:
df_test['length'] = df_test.Text.apply(leng_txt)
df_test.length.quantile(0.95)

Token indices sequence length is longer than the specified maximum sequence length for this model (681 > 512). Running this sequence through the model will result in indexing errors


41.0

In [157]:
testing_set = CustomDataset(df_test, tokenizer, MAX_LEN)
TEST_BATCH_SIZE = 32
test_params = {'batch_size': TEST_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 4
                }
testing_loader = DataLoader(testing_set, **test_params)

In [158]:
proba = torch.nn.Softmax(dim=1)
def test():
    model.eval()
    prediction = []

    with torch.no_grad():
        for i, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            #targets = data['targets'].to(device, dtype = torch.long)

            outputs = model(ids, mask, token_type_ids)
            outputs = proba(outputs[0])
            outputs = outputs.detach().cpu().numpy()
            #_, preds = torch.max(outputs, 1)
            prediction.append(outputs)
        return prediction

In [159]:
tmp = test()

In [160]:
submit = pd.DataFrame(np.vstack(tmp))

In [161]:
submit.to_csv('submit.csv', index=False)

BERT as Feautre Extractor

In [12]:
from transformers import BertModel

In [96]:
train_size = 1
train_dataset=df.sample(frac=train_size,random_state=648)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(df_test, tokenizer, MAX_LEN)
print("TEST Dataset: {}".format(df_test.shape))

FULL Dataset: (10240, 3)
TRAIN Dataset: (10240, 3)
TEST Dataset: (1267, 3)


In [97]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 4
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 4
                }

train_dataloader = DataLoader(training_set, **train_params)
test_dataloader = DataLoader(testing_set, **test_params)

In [ ]:
Bertmodel = BertModel.from_pretrained('bert-base-uncased')
Bertmodel.to(device)

In [101]:
Bertmodel.eval()
prediction = []

with torch.no_grad():
    for i, data in enumerate(train_dataloader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        #targets = data['targets'].to(device, dtype = torch.long)

        _, outputs = Bertmodel(ids, mask, token_type_ids)
        outputs = outputs.detach().cpu().numpy()
        #_, preds = torch.max(outputs, 1)
        prediction.append(outputs)

In [32]:
prediction[0].shape

(64, 768)

In [102]:
bert_features = np.vstack(prediction)

In [103]:
bert_features.shape

(1267, 768)

In [36]:
train_dataset.head()

,Labels,Text,Text_Tag
0,0,Congress decision to spend less around the wor...,congress federal-budget foreign-policy
1,0,As a result of Chris Sununus vote against a Pl...,abortion health-care voting-record
2,5,Says ending the direct payment farm subsidy pr...,agriculture federal-budget
3,3,Im also the only lieutenant governor to not ta...,state-budget
4,0,President Ronald Reagan sent troops into confl...,foreign-policy history military


In [104]:
corpus = train_dataset.Text_Tag.values

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
vectorizer = TfidfVectorizer(stop_words='english')
tf_idf = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())
print(tf_idf.shape)

(10240, 176)


In [45]:
tf_idf = tf_idf.toarray()

In [107]:
# #test
# tf_idf_test = vectorizer.transform(df_test.Text_Tag.values)
# print(tf_idf_test.shape)
# tf_idf_test = tf_idf_test.toarray()

(1267, 176)


In [108]:
# #Test
# testset = np.concatenate((bert_features, tf_idf_test), axis=1)
# testset.shape

(1267, 944)

In [82]:
X = np.concatenate((bert_features, tf_idf), axis=1)
X.shape

(10240, 944)

In [83]:
y = train_dataset.Labels.values
y.shape

(10240,)

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [85]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.utils import np_utils

In [86]:
# one-hot encoding using keras' numpy-related utilities
n_classes = 6
print("Shape before one-hot encoding: ", y_train.shape)
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", y_train.shape)

Shape before one-hot encoding:  (8192,)
Shape after one-hot encoding:  (8192, 6)


In [123]:
model = Sequential()
model.add(Dense(1024, input_shape=(944,), activation='relu'))
model.add(Dropout(0.4))
# output layer
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [124]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 1024)              967680    
_________________________________________________________________
dropout_21 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 6)                 6150      
Total params: 973,830
Trainable params: 973,830
Non-trainable params: 0
_________________________________________________________________


In [125]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
hist = model.fit(X_train, y_train,
                 batch_size=128,
                 epochs=500,
                 validation_data=(X_test, y_test),
                 )
plot_hist(hist)

In [109]:
pred_sub = model.predict(testset)
pred_sub.shape

(1267, 6)

In [110]:
pd.DataFrame(pred_sub).to_csv('submit.csv', index=False)